In [1]:
#Here i try to bulid some possible useful Gates 

In [23]:
import  numpy as np 
from math import sin ,cos

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister 
from qiskit import Aer, execute

from qiskit.aqua.algorithms import VQE, NumPyEigensolver

from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.quantum_info import Pauli

from qiskit.circuit import ParameterVector
from qiskit.aqua.components.optimizers import  SPSA

In [3]:
from tools import I,X,Y,Z
from tools import Match_gate_test

In [4]:
from tools import decompose

In [5]:
def show_mat(m):
    for i in m:
        print ('[ {}  {}  {}  {} ]'.format(i[0],i[1],i[2],i[3]))

In [6]:
#Exact solver for reference:
def Exact_solver(qubitOp):
    ex = NumPyEigensolver(qubitOp)
    result = ex.run()
    ref = result['eigenvalues']
    return np.real(ref)[0]

In [42]:
#Rx_theta matrix
def Rx_m(theta):
    return np.array([[cos(theta[0]/2),        -1.0j*sin(theta[0]/2)],
                     [-1.j*sin(theta[0]/2),   cos(theta[0]/2)]],dtype=np.complex128)
#Ry_theta matrix
def Ry_m(theta):
    return np.array([[cos(theta/2),      -sin(theta/2)],
                     [sin(theta/2),   cos(theta/2)]],dtype=np.complex128)

#Rz_theta matrix
def Rz_m(theta):
    return np.array([[exp(-1.0j*theta), 0],
                     [0,                   exp(1.0j*theta)]],dtype=np.complex128)


#Matchgate builder

def G_m(m1,m2,norm):
    return norm*np.array([[m1[0][0], 0, 0, m1[0][1]],
                 [0, m2[0][0], m2[0][1], 0],
                 [0, m2[1][0], m2[1][1],       0],
                 [m1[1][0], 0, 0, m1[1][1]]],dtype=np.complex128)

def G(m1,m2,norm,label):
    Gm=G_m(m1,m2,norm)
    G=QuantumCircuit(2,name=label)
    G.unitary(Gm,[0,1])
    return G.to_instruction()

SyntaxError: unexpected character after line continuation character (<ipython-input-42-062492e03ae0>, line 3)

In [31]:
#XX entengeler matchgate
GX_1=G_m(Rx_m(np.pi/4),Rx_m(np.pi/4),1)
print("X_1|00>=(1/sqrt(2)(|00>-i|11>)).")
show_mat(GX_1)
print("X_1 is a matchgate:",Match_gate_test(GX_1))


X_1|00>=(1/sqrt(2)(|00>-i|11>)).
[ (0.9238795325112867+0j)  0j  0j  -0.3826834323650898j ]
[ 0j  (0.9238795325112867+0j)  -0.3826834323650898j  0j ]
[ 0j  -0.3826834323650898j  (0.9238795325112867+0j)  0j ]
[ -0.3826834323650898j  0j  0j  (0.9238795325112867+0j) ]
X_1 is a matchgate: 1


In [32]:
#inside a quantum circuit
demo=QuantumCircuit(4)
demo.append(G(Rx_m(np.pi/4),Rx_m(np.pi/4),1,'GXX_ent'),[1,2])
demo.draw()


q_0: ────────────
     ┌──────────┐
q_1: ┤0         ├
     │  GXX_ent │
q_2: ┤1         ├
     └──────────┘
q_3: ────────────

In [33]:
# let's create an ansatz with GXX_1


In [34]:
#Teste Hamiltoninan
H=-1.05 * np.kron(I,I) + 0.39 * np.kron(I,Z) - 0.39 * np.kron(Z,I) - 0.01 * np.kron(Z,Z) + 0.18 * np.kron(X,X)

#Qubit Opperator
h,h_label=decompose(H)
pauli_list=[]
for i in range(len(h)):
    pauli_list.append([h[i],Pauli.from_label(h_label[i])])
HOp=WeightedPauliOperator(pauli_list) 
###########################################################################################################

for i in range(len(pauli_list)):
    if(pauli_list[i][0]!=0):
        print(pauli_list[i][1])
        print(pauli_list[i][0])
        

II
(-1.0499999999999998+0j)
IZ
(0.38999999999999996+0j)
XX
(0.18+0j)
ZI
(-0.39+0j)
ZZ
(-0.009999999999999953+0j)


In [58]:
#ansatz

ansatz = QuantumCircuit(2)

params = ParameterVector('theta', length=4*2)

    
it = iter(params)

#ansatz.append(G(Rx_m(next(it)),Rx_m(next(it)),1,'GXX_ent'),[0,1])
#ansatz.append(G(Rx_m(next(it)),Rx_m(next(it)),1,'GXX_ent'),[0,1])


ansatz.ry(next(it), 0)
ansatz.rz(next(it), 0)
ansatz.ry(next(it), 1)
ansatz.rz(next(it), 1)
ansatz.cx(0,1)
ansatz.ry(next(it), 0)
ansatz.rz(next(it), 0)
ansatz.ry(next(it), 1)
ansatz.rz(next(it), 1)


ansatz.draw()

┌──────────────┐┌──────────────┐     ┌──────────────┐┌──────────────┐
q_0: ┤ RY(theta[0]) ├┤ RZ(theta[1]) ├──■──┤ RY(theta[4]) ├┤ RZ(theta[5]) ├
     ├──────────────┤├──────────────┤┌─┴─┐├──────────────┤├──────────────┤
q_1: ┤ RY(theta[2]) ├┤ RZ(theta[3]) ├┤ X ├┤ RY(theta[6]) ├┤ RZ(theta[7]) ├
     └──────────────┘└──────────────┘└───┘└──────────────┘└──────────────┘

In [21]:
params[1]

Parameter(theta[1])

In [46]:
param = ParameterVector('theta', length=4*2)
i = iter(param)
print(param)
print(i)

theta, ['theta[0]', 'theta[1]', 'theta[2]', 'theta[3]', 'theta[4]', 'theta[5]', 'theta[6]', 'theta[7]']


In [52]:
param._params

[Parameter(theta[0]),
 Parameter(theta[1]),
 Parameter(theta[2]),
 Parameter(theta[3]),
 Parameter(theta[4]),
 Parameter(theta[5]),
 Parameter(theta[6]),
 Parameter(theta[7])]

In [45]:
cos(param[0])

TypeError: ParameterExpression with unbound parameters ({Parameter(theta[0])}) cannot be cast to a float.

In [ ]:
optimizer=SPSA(200)
backend = Aer.get_backend('statevector_simulator')

In [ ]:
vqe = VQE(HOp, ansatz, optimizer)
result = vqe.run(backend)
print('VQE:',result.eigenvalue.real ,'Reference:', Exact_solver(HOp))

In [33]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright